In [3]:
# 必要なライブラリ
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb
import lightgbm as lgb

from sklearn.metrics import roc_curve, roc_auc_score

In [4]:
# データの読み込み
df = pd.read_csv("dataset/PAS_table.csv")

# 予めシャッフルしておく
df = df.sample(frac=1)
df = df.reset_index()

In [6]:
df = df.drop("index", axis=1)

In [8]:
#df_id = df["ID"]
X = np.array(df.drop(["PAS", "Week"], axis=1))
t = np.array(df["PAS"])

In [20]:
import matplotlib.pyplot as plt

# クラスの割合が一定になるように5分割
skf = StratifiedKFold(n_splits=5, shuffle=True)

tp = []
tn = []
fp = []
fn = []

coefs = []

for i, (train_index, test_index) in enumerate(skf.split(X, t)):
    ids = X[:, 0]
    train_X = np.delete(X, 0, axis=1)  
    
    #model = LogisticRegression()
    #model = RandomForestClassifier()
    model = lgb.LGBMClassifier()

    
    model.fit(train_X[train_index], t[train_index])
    y_pred = model.predict(train_X[test_index])
    
    fpr, tpr, thresholds = roc_curve(t[test_index], y_pred)
    auc = roc_auc_score(t[test_index], y_pred)
    plt.figure()
    plt.plot(fpr, tpr, label="ROC (AUC=" + str(round(auc, 4)) + ")" )
    plt.xlabel("False positive rate")
    plt.ylabel("True positive rate")
    plt.legend()
    plt.savefig("result/clinical_feature_ROC_" + str(i+1) + ".png")
    plt.clf()

    
    #coefs.append(model.coef_)
    
    kth_test_ids = ids[test_index]
    
    for i in range(len(y_pred)):
        if y_pred[i] == 1 and t[test_index][i] == 1:
            tp.append(kth_test_ids[i])
        elif y_pred[i] == 1 and t[test_index][i] == 0:
            fp.append(kth_test_ids[i])
        elif y_pred[i] == 0 and t[test_index][i] == 0:
            tn.append(kth_test_ids[i])
        else:
            fn.append(kth_test_ids[i])

acc = (len(tp) + len(tn)) / (len(tp) + len(tn) + len(fp) + len(fn))
sen = len(tp) / (len(fn) + len(tp))
spec = len(tn) / (len(fp) + len(tn)) 

print("Accuracy: {} \nSensitivity: {} \nSpecificity: {} \nAUC: {}".format(acc, sen, spec, auc))

Accuracy: 0.8232558139534883 
Sensitivity: 0.6181818181818182 
Specificity: 0.89375 
AUC: 0.8778409090909091


<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [150]:
coefs

[array([[-0.03980314, -0.12427195,  0.36278391,  0.40301693,  2.47052321,
          0.26184445]]),
 array([[-0.03086225,  0.01346169,  0.29046752,  0.42621771,  2.65822672,
          0.37934514]]),
 array([[-4.20210809e-02,  4.72193431e-02,  2.17647994e-01,
          4.89439831e-01,  3.07496187e+00,  1.22440768e-03]]),
 array([[-0.03228012,  0.06196481,  0.05640474,  0.72463738,  2.69403282,
          0.31449853]]),
 array([[-0.01920085,  0.05264623,  0.0847624 ,  0.53412299,  2.66339469,
          0.38636527]])]

In [12]:
t[test_index]

array([1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0])

In [2]:
train_X

NameError: name 'train_X' is not defined